### Where does the dataset come from? Download from yelp.com

In [ ]:
reviews = get_file('tables/reviews_pennsylvania.csv')

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('spacytextblob')

def check_polarity(text):
    doc = nlp(text)
    return doc._.blob.polarity

reviews['polarity'] = reviews['text'][:10000].apply(lambda x: check_polarity(x))

In [ ]:
with open('tables/reviews_polarity.csv', 'w') as f:
    reviews[:10000].to_csv(f, header=True, index=False)

In [ ]:
reviews = get_file('tables/reviews_polarity.csv')

In [ ]:
negative_reviews = reviews[reviews['polarity'] < 0]

### Analys negative reviews

#### We want to find statements about food

In [72]:
# Training the entity rule to recognize food

food = pd.read_csv('food_labels/food.csv')

In [89]:
food_labels = food[food['description'].str.contains('[a-zA-Z]') == True]['description']

In [90]:
food_labels = food_labels[food_labels.str.split().apply(len) <= 2].drop_duplicates()

In [91]:
food_labels = food_labels.str.lower()

In [92]:
food_labels = food_labels[food_labels.str.contains('.*,.*,.*', regex=True) == False]

In [93]:
# Fix: Foods are comma seperated and order is reversed: 'muffins, blueberry' become 'blueberry muffins'

food_labels[food_labels.str.contains(', ') == True] = (
    food_labels[food_labels.str.contains(', ') == True].str.split(', ', expand=True)[1] +
    ' ' +
    food_labels[food_labels.str.contains(', ') == True].str.split(', ', expand=True)[0]
)

food_labels[food_labels.str.contains(',') == True] = (
    food_labels[food_labels.str.contains(',') == True].str.split(',', expand=True)[1] +
    ' ' +
    food_labels[food_labels.str.contains(',') == True].str.split(',', expand=True)[0]
)

In [94]:
# If label has no plural, create singular form. If label has no singular, create plural 

import spacy
import lemminflect

nlp = spacy.load('en_core_web_lg')

inflected_labels = []

for label in food_labels:
    
    doc = nlp(label)

    if len(doc) == 1:
        if doc[0].tag_ == 'NNS':
            inflected_labels.append(doc[0]._.inflect('NN'))
        else:
            inflected_labels.append(doc[0]._.inflect('NNS'))

    if len(doc) == 2:
        if doc[1].tag_ == 'NNS':
            inflected_labels.append(doc[0].text + ' ' + doc[1]._.inflect('NN'))
        else:
            inflected_labels.append(doc[0].text + ' ' + doc[1]._.inflect('NNS'))

In [99]:
food_labels_inflected = pd.Series(inflected_labels)

In [100]:
food_labels = pd.concat([food_labels, food_labels_inflected])

In [101]:
food_labels.drop_duplicates(inplace=True)

In [102]:
food_labels[food_labels.str.contains('bar', regex=True)]

2645        italian barley
3702        barbecue spice
5431               bar bar
5547            fudge bars
7690         sweet bar-b-q
               ...        
15881          chunky bars
16540           limon bars
16663         hazelnut bar
16778    blueberry golbars
17113             paleobar
Length: 538, dtype: object

In [103]:
# Add food labels to entity ruler

food_labels = food_labels  # Remove 'bar' from foods_labels, add 'product'

patterns = []

nlp = spacy.load('en_core_web_lg')

ruler = nlp.add_pipe('entity_ruler', before='ner')

for label in food_labels:
    patterns.append({'label': 'FOOD', 'pattern': label})

ruler.add_patterns(patterns)

In [104]:
len(patterns)

30704

In [106]:
ruler.to_disk('food_labels/food_patterns.jsonl')

In [1]:
nlp = spacy.load('en_core_web_lg')

ruler = nlp.add_pipe("entity_ruler", before='ner')

ruler.from_disk('food_labels/food_patterns.jsonl')

NameError: name 'spacy' is not defined